In [83]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import urllib

In [84]:
import sys
sys.path.append("../")
from consts import CLEAN_SCRAPE_DIR, BEFORE_2010_DIR, IN_2010S, AFTER_2020

In [85]:
TARGET_DIR = CLEAN_SCRAPE_DIR

In [86]:
df = pd.read_csv(f"./metadata/{TARGET_DIR}/meta_en_prime.csv", encoding='cp1252', header=None)
# USEFUL ENCODINGS: cp1252, utf-8

In [87]:
df.tail()

,0,1,2
6379,Cabin 28,/Cabin-28-Lee-Bane/dp/B079WDP9CP/ref=sr_1_1580...,2017|NR|CC
6380,The Perfect Victim,/Perfect-Victim-Elizabeth-Rohrbaugh/dp/B07749H...,2013|CC
6381,EVANGELION:3.33 YOU CAN (NOT) REDO.,/EVANGELION-3-33-YOU-CAN-REDO/dp/B08SVTV9DQ/re...,2013|CC
6382,God's Miracles,/Gods-Miracles-Bryant-Gumbel/dp/B0763TBQXY/ref...,1982|CC
6383,Philomena,/Philomena-Judi-Dench/dp/B00IIVK3ZU/ref=sr_1_1...,2013|PG-13|CC


In [88]:
df.columns=['title', 'link', 'tags']

In [89]:
df.shape

(6384, 3)

In [90]:
df['year'] = pd.to_numeric(df['tags'].str.split('|').str[0], errors='coerce')

### First, we'll convert the name to only alphanumeric characters

In [91]:
df['clean_title'] = [unidecode(text) for text in df['title']]
df['clean_title'] = df['clean_title'].str.replace("[^0-9a-zA-Z ]", "", regex=True).str.replace(" ", "_")

In [92]:
df.head()

,title,link,tags,year,clean_title
0,My Fault,/My-Fault-Nicole-Wallace/dp/B0B683GB78/ref=sr_...,2023|CC,2023.0,My_Fault
1,Jurassic World Dominion,/Jurassic-World-Dominion-Chris-Pratt/dp/B0B5NN...,2022|PG-13|CC,2022.0,Jurassic_World_Dominion
2,Top Gun: Maverick,/Top-Gun-Maverick-Tom-Cruise/dp/B0B214XZB4/ref...,2022|PG-13|CC,2022.0,Top_Gun_Maverick
3,Creed III,/Creed-III-Michael-B-Jordan/dp/B0B8TKJCRQ/ref=...,2023|PG-13|CC,2023.0,Creed_III
4,Sonic the Hedgehog 2,/Sonic-Hedgehog-2-James-Marsden/dp/B09ZRVBZG7/...,2022|PG|CC,2022.0,Sonic_the_Hedgehog_2


We'll also add the index to make the names unique.

In [93]:
df['fname'] = pd.Series(np.arange(0, df.shape[0]), dtype='str') + "_" + df['clean_title']

In [94]:
df.head()

,title,link,tags,year,clean_title,fname
0,My Fault,/My-Fault-Nicole-Wallace/dp/B0B683GB78/ref=sr_...,2023|CC,2023.0,My_Fault,0_My_Fault
1,Jurassic World Dominion,/Jurassic-World-Dominion-Chris-Pratt/dp/B0B5NN...,2022|PG-13|CC,2022.0,Jurassic_World_Dominion,1_Jurassic_World_Dominion
2,Top Gun: Maverick,/Top-Gun-Maverick-Tom-Cruise/dp/B0B214XZB4/ref...,2022|PG-13|CC,2022.0,Top_Gun_Maverick,2_Top_Gun_Maverick
3,Creed III,/Creed-III-Michael-B-Jordan/dp/B0B8TKJCRQ/ref=...,2023|PG-13|CC,2023.0,Creed_III,3_Creed_III
4,Sonic the Hedgehog 2,/Sonic-Hedgehog-2-James-Marsden/dp/B09ZRVBZG7/...,2022|PG|CC,2022.0,Sonic_the_Hedgehog_2,4_Sonic_the_Hedgehog_2


In [95]:
df[df['title'].duplicated()]

,title,link,tags,year,clean_title,fname
867,Blackbird,/Blackbird-Susan-Sarandon/dp/B09P1MH4V7/ref=sr...,2020|R|CC,2020.0,Blackbird,867_Blackbird
1250,Global Meltdown,/Global-Meltdown-Michael-Pare/dp/B0B66791WP/re...,2018|TV-MA|CC,2018.0,Global_Meltdown,1250_Global_Meltdown
1413,The Lost World,/Lost-World-John-Rhys-Davies/dp/B077YZHZYD/ref...,1992|CC,1992.0,The_Lost_World,1413_The_Lost_World
1582,Payback,/Payback-Matt-Levett/dp/B08TLP11CP/ref=sr_1_30...,2021|CC,2021.0,Payback,1582_Payback
2009,His Girl Friday,/His-Girl-Friday-Cary-Grant/dp/B099D8SF1F/ref=...,1940|TV-14|CC,1940.0,His_Girl_Friday,2009_His_Girl_Friday
...,...,...,...,...,...,...
6146,Goodbye World,/Goodbye-World-Adrian-Grenier/dp/B0755CXJ95/re...,2014|TV-14|CC,2014.0,Goodbye_World,6146_Goodbye_World
6198,One Body Too Many,/One-Body-Many-Jack-Haley/dp/B0B8X5WR6X/ref=sr...,2023|CC,2023.0,One_Body_Too_Many,6198_One_Body_Too_Many
6216,Jigsaw,/Jigsaw-4K-UHD-Hannah-Anderson/dp/B07WHL775C/r...,2017|R|CC,2017.0,Jigsaw,6216_Jigsaw
6282,Abandoned,/Abandoned-Dominic-Purcell/dp/B076279G5R/ref=s...,2016|CC,2016.0,Abandoned,6282_Abandoned


There could be multiple files with the same name. I checked tags as well. Same movies can have slightly different tags (years) too. But if the short_url matches, then it is likely a duplicate.

In [96]:
df[df['title'] == "No Surrender"]

,title,link,tags,year,clean_title,fname
4372,No Surrender,/No-Surrender-Mena-Suvari/dp/B08XVTSDX8/ref=sr...,2012|TV-14|CC,2012.0,No_Surrender,4372_No_Surrender
4769,No Surrender,/No-Surrender-Brandon-Hayes/dp/B074MJJFHF/ref=...,2013|CC,2013.0,No_Surrender,4769_No_Surrender


In [97]:
df['short_url'] = df['link'].str.extract("(/[^/]*/)")

Some can also have the same short_url, but could be different movies. We can check the title as well along with the short_url for validation like Doc Wst - Part 1 and Doc West - Part 2 in com dataset.

In [98]:
df[df['short_url'].isin(df[df[['short_url']].duplicated()]['short_url'])].sort_values(by='short_url', ascending=True).to_csv("test.csv")

In [99]:
df[df["short_url"] == "/Nishabdham-Telugu-Anushka-Shetty/"]

,title,link,tags,year,clean_title,fname,short_url


In [102]:
dups = df[df[['title', 'year']].duplicated()]

In [103]:
dups.shape

(37, 7)

In [117]:
df = pd.read_csv("test.csv")
df[~df['title'].isin(df[~df[['short_url', 'year']].duplicated()]['title'])]

,Unnamed: 0,title,link,tags,year,clean_title,fname,short_url
1,3960,"""1/1""",/1-Lindsey-Shaw/dp/B0B8JJW2SY/ref=sr_1_5129?qi...,2018|TV-MA|CC,2018.0,11,3960_11,/1-Lindsey-Shaw/
3,3818,1900,/1900-Robert-Niro/dp/B07RF9PNDY/ref=sr_1_5003?...,1977|R|CC,1977.0,1900,3818_1900,/1900-Robert-Niro/
11,2242,Angel And The Badman,/Angel-Badman-John-Wayne/dp/B09KGHXVQ9/ref=sr_...,1947|CC,1947.0,Angel_And_The_Badman,2242_Angel_And_The_Badman,/Angel-Badman-John-Wayne/
12,843,Angel And The Badman,/Angel-Badman-John-Wayne/dp/B099YLPFLK/ref=sr_...,1947|CC,1947.0,Angel_And_The_Badman,843_Angel_And_The_Badman,/Angel-Badman-John-Wayne/
30,5704,Cartoon Classics - Vol. 3: 25 Favorite Cartoon...,/Cartoon-Classics-Favorite-Cartoons-Hours/dp/B...,2017|PG-13|CC,2017.0,Cartoon_Classics__Vol_3_25_Favorite_Cartoons__...,5704_Cartoon_Classics__Vol_3_25_Favorite_Carto...,/Cartoon-Classics-Favorite-Cartoons-Hours/
34,3855,The Competition,/Competition-Thora-Birch/dp/B07843WHHK/ref=sr_...,2018|TV-14|CC,2018.0,The_Competition,3855_The_Competition,/Competition-Thora-Birch/
50,4633,Howard The Duck,/Howard-Duck-Lea-Thompson/dp/B0C2SQJFZQ/ref=sr...,1986|PG|CC,1986.0,Howard_The_Duck,4633_Howard_The_Duck,/Howard-Duck-Lea-Thompson/
62,4327,By Night’s End,/Nights-End-Michelle-Rose/dp/B08G88RRHV/ref=sr...,2020|NR|CC,2020.0,By_Nights_End,4327_By_Nights_End,/Nights-End-Michelle-Rose/
64,534,Nursery Rhymes and Kids Songs Vol 2 - All Babi...,/Nursery-Rhymes-Kids-Songs-Vol/dp/B0B8N8MWMR/r...,2022|CC,2022.0,Nursery_Rhymes_and_Kids_Songs_Vol_2__All_Babie...,534_Nursery_Rhymes_and_Kids_Songs_Vol_2__All_B...,/Nursery-Rhymes-Kids-Songs-Vol/


Let's check some of them.

In [48]:
# for url in dups['short_url']:
#     print(df.loc[df['short_url'] == url])
#     input()

These are some cases: The Little Princess, two dates, but newer one is just remastered
Virginia, listed as two entries, but are same movies.
There were many such cases, but seems like checking for short_url and title is a good heuristic to remove duplicates.

In [49]:
df_clean = df[~df[['title', 'year']].duplicated()]

We removed entries with duplicate title and year.

We'll also remove entries with duplicate title and short_url next.

In [50]:
df_clean = df_clean[~df_clean[['title', 'short_url']].duplicated()]

In [51]:
df.shape, df_clean.shape

((93, 9), (61, 9))

However, let's check the entries with duplicate titles.

In [52]:
df_clean[df_clean['title'] == "Bad Cupid"]

,Unnamed: 0.1,Unnamed: 0,title,link,tags,year,clean_title,fname,short_url


In [53]:
df_clean[df_clean['title'].duplicated()]

,Unnamed: 0.1,Unnamed: 0,title,link,tags,year,clean_title,fname,short_url


In [54]:
# df_clean[df_clean['title'] == "Chasing"]

For some cases like above, we need to unquote the url and unidecode. (Some characters with accents could be removed by pandas which will cause issues in matching later)

In [55]:
df_clean_copy = df_clean.copy()
df_clean_copy['clean_short_url'] = [unidecode(urllib.parse.unquote(string)) for string in df_clean_copy['short_url']]

In [56]:
df_clean_copy.head()

,Unnamed: 0.1,Unnamed: 0,title,link,tags,year,clean_title,fname,short_url,clean_short_url
0,0,4729,1/1,/1-Lindsey-Shaw/dp/B07DCXF18Q/ref=sr_1_5834?qi...,2018|TV-MA|CC,2018.0,11,4729_11,/1-Lindsey-Shaw/,/1-Lindsey-Shaw/
1,1,3960,"""1/1""",/1-Lindsey-Shaw/dp/B0B8JJW2SY/ref=sr_1_5129?qi...,2018|TV-MA|CC,2018.0,11,3960_11,/1-Lindsey-Shaw/,/1-Lindsey-Shaw/
2,2,4076,1900: Extended,/1900-Robert-Niro/dp/B014R1VMA4/ref=sr_1_5245?...,1977|R|CC,1977.0,1900_Extended,4076_1900_Extended,/1900-Robert-Niro/,/1900-Robert-Niro/
3,3,3818,1900,/1900-Robert-Niro/dp/B07RF9PNDY/ref=sr_1_5003?...,1977|R|CC,1977.0,1900,3818_1900,/1900-Robert-Niro/,/1900-Robert-Niro/
4,4,6282,Abandoned,/Abandoned-Dominic-Purcell/dp/B076279G5R/ref=s...,2016|CC,2016.0,Abandoned,6282_Abandoned,/Abandoned-Dominic-Purcell/,/Abandoned-Dominic-Purcell/


Let's check for duplicates.

In [57]:
df_clean_copy[df_clean_copy[['title', 'clean_short_url']].duplicated()]

,Unnamed: 0.1,Unnamed: 0,title,link,tags,year,clean_title,fname,short_url,clean_short_url


There was only one such entry, so we'll just remove it.

In [58]:
df_clean_copy = df_clean_copy[~df_clean_copy[['title', 'clean_short_url']].duplicated()]

In [59]:
df_clean_copy.shape

(61, 10)

In [60]:
df_clean_copy.to_csv(f"./metadata/{TARGET_DIR}/clean_meta_en_prime.csv", index=False)